In [22]:
from scipy.cluster.vq import *
from sklearn.preprocessing import StandardScaler
from sklearn.svm import LinearSVC, SVC
from sklearn.externals import joblib
from sklearn.model_selection import train_test_split, cross_val_score, cross_validate
from sklearn.metrics import confusion_matrix
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import BaggingClassifier
import matplotlib.pyplot as plt
import cv2
import numpy as np
import os
from imutils import paths
import pickle
from sklearn.decomposition import PCA
from sklearn.metrics import accuracy_score
from skimage.feature import hog
from skimage.feature import local_binary_pattern
from skimage import filters
import pandas as pd

In [2]:
def openfile(file):
    f = open(file,"r", encoding='utf-8')
    lines = f.readlines()

    image_paths=[]  
    image_classes=[]
    for line in lines:
        curLine=line.strip().split(" ")
        image_paths.append(curLine[0]) 
        image_classes.append(curLine[-1])

    time=1
    allt=len(image_paths)
    ims = []
    print("loading images")
    for image_path in image_paths:
        img  = cv2.imread(image_path)
        img=cv2.cvtColor(img,cv2.COLOR_BGR2GRAY)
        img=cv2.resize(img,(64,64),interpolation=cv2.INTER_CUBIC)
        ims.append(img)
        print(str(time)+"/"+str(allt) +"  " + image_path, end="\r")
        time = time +1
    print("\nDone!")
    return(ims,image_paths,image_classes)


def siftfeature(image_paths,image_classes,ims):
    sift= cv2.xfeatures2d.SIFT_create()
    des_list = []
    time=1
    allt=len(image_paths)
    print("Calculating all the descriptors and keypoints...")
    for im in ims:
        kp, des = sift.detectAndCompute(im,None)
        if str(des) != 'None':
            des_list.append((image_paths[time-1], des))       
            print(str(time) + "/" + str(allt), end="\r")
            time = time +1
        else:
            print(image_paths[time-1])
            del image_classes[time-1]
            del image_paths[time-1]
    print("\nDone!")
    print("image_classes:%d",len(image_classes))
    print("image_paths:%d",len(image_paths))
    
    time=1
    allt=len(des_list)
    descriptors = des_list[0][1]
    for image_path, descriptor in des_list[1:]:
        descriptors = np.vstack((descriptors, descriptor))
        print(str(time) + "/" + str(allt), end="\r")
        time = time +1
    
    k = 128
    voc, variance = kmeans(descriptors, k, 1)
    
    im_features = np.zeros((len(image_paths), k), "float32")
    for i in range(len(image_paths)):
        print("Calculating distance for image "+str(i)+"...", end="\r")
        words, distance = vq(des_list[i][1],voc)
        for w in words:
            im_features[i][w] += 1
            
    nbr_occurences = np.sum( (im_features > 0) * 1, axis = 0)
    idf = np.array(np.log((1.0*len(image_paths)+1) / (1.0*nbr_occurences + 1)), 'float32')
    
    stdSlr = StandardScaler().fit(im_features)
    im_features = stdSlr.transform(im_features)
    
    return(im_features,image_classes)

def lbpfeature(image_paths,image_classes,ims):
    des_list = []
    time=1
    allt=len(image_paths)
    print("Calculating all the descriptors and keypoints...")
    for im in ims:
        lbp=local_binary_pattern(im, 8, 1, 'default')
        max_bins = int(lbp.max() + 1)
        des, _ = np.histogram(lbp, normed=True, bins=max_bins, range=(0, max_bins))
        if str(des) != 'None':
            des_list.append(des)
            print(str(time) + "/" + str(allt), end="\r")
            time = time +1
        else:
            print(image_paths[time-1])
            del image_classes[time-1]
            del image_paths[time-1]
    print("\nDone!")
    
    return(des_list,image_classes)

def hogfeature(image_paths,image_classes,ims):
    des_list = []
    time=1
    allt=len(image_paths)
    print("Calculating all the descriptors and keypoints...")
    for im in ims:
        hog_image,des = hog(im, orientations=4, pixels_per_cell=(8,8),cells_per_block=(2, 2),block_norm= 'L2',visualise=True)
        if str(des) != 'None':
            des_list.append((image_paths[time-1], des))
            print(str(time) + "/" + str(allt), end="\r")
            time = time +1
        else:
            print(image_paths[time-1])
            del image_classes[time-1]
            del image_paths[time-1]
    print("\nDone!")
    print("image_classes:%d",len(image_classes))
    print("image_paths:%d",len(image_paths))
    
    time=1
    allt=len(des_list)
    descriptors = des_list[0][1]
    for image_path, descriptor in des_list[1:]:
        descriptors = np.vstack((descriptors, descriptor))
        print(str(time) + "/" + str(allt), end="\r")
        time = time +1
    
    k = 128
    voc, variance = kmeans(descriptors, k, 1)
    
    im_features = np.zeros((len(image_paths), k), "float32")
    for i in range(len(image_paths)):
        print("Calculating distance for image "+str(i)+"...", end="\r")
        words, distance = vq(des_list[i][1],voc)
        for w in words:
            im_features[i][w] += 1
            
    nbr_occurences = np.sum( (im_features > 0) * 1, axis = 0)
    idf = np.array(np.log((1.0*len(image_paths)+1) / (1.0*nbr_occurences + 1)), 'float32')
    
    stdSlr = StandardScaler().fit(im_features)
    im_features = stdSlr.transform(im_features)
    
    return(im_features,image_classes)

def sobelfeature(image_paths,image_classes,ims):
    des_list = []
    time=1
    allt=len(image_paths)
    print("Calculating all the descriptors and keypoints...")
    for im in ims:
        des=filters.sobel(im)
        if str(des) != 'None':
            des=des.flatten()
            des_list.append(des)
            print(str(time) + "/" + str(allt), end="\r")
            time = time +1
        else:
            print(image_paths[time-1])
            del image_classes[time-1]
            del image_paths[time-1]
    print("\nDone!")
    
    return(des_list,image_classes)

def predicts(clf,im_feature):
    predictions=clf.predict(im_feature)
    
    return predictions

In [3]:
siftclf= joblib.load("sift.pkl")
lbpclf= joblib.load("lbp.pkl")
hogclf= joblib.load("hog.pkl")
sobelclf= joblib.load("sobel.pkl")

In [4]:
print("start sift test")
imstest=[]
image_pathstest=[]
image_classestest=[]
imstest,image_pathstest,image_classestest=openfile("test.txt")
test_sift_features,sift_image_classestest=siftfeature(image_pathstest,image_classestest,imstest)

start sift test
loading images
450/450  images/n02172182/n02172182_981.JPEGG
Done!
Calculating all the descriptors and keypoints...
images/n02120079/n02120079_9750.JPEG
449/450
Done!
image_classes:%d 449
image_paths:%d 449


In [5]:
print("start lbp test")
imstest=[]
image_pathstest=[]
image_classestest=[]
imstest,image_pathstest,image_classestest=openfile("test.txt")
test_lbp_features,lbp_image_classestest=lbpfeature(image_pathstest,image_classestest,imstest)
pca=PCA(n_components=128)
test_lbp_features=pca.fit_transform(test_lbp_features)

start lbp test
loading images
450/450  images/n02172182/n02172182_981.JPEGG
Done!
Calculating all the descriptors and keypoints...


c:\users\user\anaconda3\envs\tensorflow1\lib\site-packages\ipykernel_launcher.py:81: VisibleDeprecationWarning: Passing `normed=True` on non-uniform bins has always been broken, and computes neither the probability density function nor the probability mass function. The result is only correct if the bins are uniform, when density=True will produce the same result anyway. The argument will be removed in a future version of numpy.


450/450
Done!


In [6]:
print("start hog test")
imstest=[]
image_pathstest=[]
image_classestest=[]
imstest,image_pathstest,image_classestest=openfile("test.txt")
test_hog_features,hog_image_classestest=hogfeature(image_pathstest,image_classestest,imstest)

start hog test
loading images
450/450  images/n02172182/n02172182_981.JPEGG
Done!
Calculating all the descriptors and keypoints...


c:\users\user\anaconda3\envs\tensorflow1\lib\site-packages\skimage\feature\_hog.py:239: skimage_deprecation: Argument `visualise` is deprecated and will be changed to `visualize` in v0.16
  'be changed to `visualize` in v0.16', skimage_deprecation)


450/450
Done!
image_classes:%d 450
image_paths:%d 450


In [7]:
print("start sobel test")
imstest=[]
image_pathstest=[]
image_classestest=[]
imstest,image_pathstest,image_classestest=openfile("test.txt")
test_sobel_features,sobel_image_classestest=sobelfeature(image_pathstest,image_classestest,imstest)
pca=PCA(n_components=128)
test_sobel_features=pca.fit_transform(test_sobel_features)

start sobel test
loading images
450/450  images/n02172182/n02172182_981.JPEGG
Done!
Calculating all the descriptors and keypoints...
450/450
Done!


In [8]:
siftpredictions=predicts(siftclf,test_sift_features)
lbppredictions=predicts(lbpclf,test_lbp_features)
hogpredictions=predicts(hogclf,test_hog_features)
sobelpredictions=predicts(sobelclf,test_sobel_features)

In [23]:
acc_sift=round((accuracy_score(sift_image_classestest, siftpredictions)),3)
acc_lbp=round((accuracy_score(lbp_image_classestest, lbppredictions)),3)
acc_hog=round((accuracy_score(hog_image_classestest, hogpredictions)),3)
acc_sobel=round((accuracy_score(sobel_image_classestest, sobelpredictions)),3)
dic={"sift":acc_sift,"lbp":acc_lbp,"hog":acc_hog,"sobel":acc_sobel}
id=["acc"]
df=pd.DataFrame(dic,index=id)

In [24]:
print(df)

       hog    lbp   sift  sobel
acc  0.018  0.013  0.024  0.031
